# Tutorial 3: How simulations define your predictions
The inverse problem has no unique solution as it is ill-posed. In order to solve it we need to constraint the space of possible solutions. While inverse solutions like minimum-norm estimates have an explicit constraint of minimum-energy, the constraints with esinet are implicit and mostly shaped by the simulations.

This tutorial aims the relation between simulation parameters and predictions.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# import mne
# import numpy as np
# from copy import deepcopy
# import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [2]:
info = get_info(sfreq=100)
fwd = create_forward_model(sampling="ico3", info=info)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.7s remaining:    1.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished


# Extent

## Simulate

In [57]:
n_samples = 1000
settings = dict(duration_of_trial=0.20, extents=(1,20), number_of_sources=1)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

Simulating data based on sparse patches.


100%|██████████| 1000/1000 [00:00<00:00, 17591.20it/s]


source data shape:  (1284, 20) (1284, 20)


100%|██████████| 1000/1000 [00:02<00:00, 387.88it/s]


## Create Data

In [58]:
import numpy as np
X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
y = np.squeeze(np.stack([src.data for src in sim.source_data]))
y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)

X = np.swapaxes(X, 1,2)
y = np.swapaxes(y, 1,2)
print(X.shape, y.shape)

(1000, 20, 61) (1000, 20, 1284)


## Build and Train

In [17]:
def sparsity(y_true, y_pred):
    return K.mean(K.square(y_pred)) / K.max(K.square(y_pred))
def custom_loss():
    def loss(y_true, y_pred):
        loss1 = tf.keras.losses.CosineSimilarity()(y_true, y_pred)
        loss2 = sparsity(None, y_pred)
        return loss1 + loss2 * 1e-3
    return loss

InvalidArgumentError: in user code:

    File "C:\Users\Lukas\AppData\Local\Temp/ipykernel_11680/1882078065.py", line 26, in None  *
        loss(x[0], x[1])
    File "C:\Users\Lukas\AppData\Local\Temp/ipykernel_11680/1882078065.py", line 14, in loss  *
        dist_new = dist[idc_true, idc_pred]

    InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [75,2] != values[1].shape = [15228,2] [Op:Pack] name: stack


In [80]:
import tensorflow as tf
import tensorflow.keras.backend as K
from scipy.spatial.distance import cdist

def chamfer(pos, thresh=0.1, dtype=tf.float32):
    dist = tf.cast(cdist(pos, pos), dtype=dtype)
    pos = tf.cast(pos, dtype=dtype)
    def loss_batch(y_true, y_pred):
        def loss(y_true, y_pred):
            # print("third: ", tf.shape(y_true))
        
            # print(y_true, y_pred)
            # find indices above threshold
            # idc_true = tf.where(K.abs(y_true) > K.max(K.abs(y_true)) * thresh)[:, 0]
            # idc_pred = tf.where(K.abs(y_pred) > K.max(K.abs(y_pred)) * thresh)[:, 0]
            
            idc_true = tf.where(K.abs(y_true) > K.mean(K.abs(y_true)) )[:, 0]
            idc_pred = tf.where(K.abs(y_pred) > K.mean(K.abs(y_pred)) )[:, 0]
            num_true_points = tf.cast(tf.shape(idc_true), dtype=dtype)
            # print(idc_true, idc_pred)
            # retrieve the correct distances
            dist_true = tf.gather(dist, idc_true, axis=0)
            dist_true = tf.gather(dist_true, idc_pred, axis=1)
            
            # print(dist_true)
            
            lowest_dists_1 = tf.reduce_min(dist_true, axis=0)
            lowest_dists_2 = tf.reduce_min(dist_true, axis=1)

            sum_squares_1 = K.sum(K.square(lowest_dists_1))
            sum_squares_2 = K.sum(K.square(lowest_dists_2))


            error = (sum_squares_1 + sum_squares_2) / 2
            error =  K.sqrt(error / num_true_points)
            # print("error on single sample and time: ", error)
            return error
        # reshaping
        new_shape = (tf.shape(y_true)[0]*tf.shape(y_true)[1], tf.shape(y_true)[2])
        y_true = tf.reshape(y_true, new_shape)
        y_pred = tf.reshape(y_pred, new_shape)
        # print(y_true, y_pred)
        batched_losses = tf.map_fn(lambda x:
                                    loss(x[0], x[1]),
                                    (y_true, y_pred), dtype=tf.float32)
        error = K.mean(tf.stack(batched_losses))
        # print("error on all samples and all times: ", error)
        return error
    return loss_batch

import tensorflow as tf
thresh=0.1, 
dtype=tf.float32

batch_size = 32
n_time, n_dipoles = (25, 1284)
source_idc = np.array([0,])
y_true = np.zeros((batch_size, n_time, n_dipoles))
y_true[:, :, source_idc] = 1
y_pred = np.zeros((batch_size, n_time, n_dipoles))
y_pred[:, :, 2] = 1

L = chamfer(pos)
L(y_true, y_pred)



<tf.Tensor: shape=(), dtype=float32, numpy=32.393356>

In [79]:
np.sqrt(np.sum((pos[0]-pos[2])**2))

32.393355459931385

# LSTM

In [60]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, TimeDistributed, Bidirectional, LSTM, GRU, multiply, Activation, Dropout
from tensorflow.keras.regularizers import l1

leadfield, pos = util.unpack_fwd(fwd)[1:3]
n_channels, n_dipoles = leadfield.shape
input_shape = (None, None, n_channels)
tf.keras.backend.set_image_data_format('channels_last')

n_dense_units = 150
n_lstm_units = 128
activation_function = "tanh"
batch_size = 32
epochs = 100
dropout = 0.1

def threshold_activation(x):
    return tf.cast(x > 0.3, dtype=tf.float32)

inputs = tf.keras.Input(shape=(None, n_channels), name='Input')
fc1 = TimeDistributed(Dense(n_dense_units, 
            activation=activation_function), 
            name='FC1')(inputs)
fc1 = Dropout(dropout)(fc1)


lstm1 = Bidirectional(GRU(n_lstm_units, return_sequences=True, 
            input_shape=(None, n_dense_units), dropout=dropout), 
            name='LSTM1')(inputs)
mask = TimeDistributed(Dense(n_dense_units, 
            activation="sigmoid"), 
            name='Mask')(lstm1)
# mask = Activation(threshold_activation)(mask)

multi = multiply([fc1, mask], name="multiply")
final_out = TimeDistributed(Dense(n_dipoles, 
            activation="linear"),
            name='FC2')(multi)
model = tf.keras.Model(inputs=inputs, outputs=final_out, name='Contextualizer')


# model.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer="adam")
# model.compile(loss=custom_loss(), optimizer="adam", metrics=[tf.keras.losses.CosineSimilarity(), sparsity])
model.compile(loss=chamfer(pos), optimizer="adam", metrics=[tf.keras.losses.CosineSimilarity(),])


model.summary()
model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Model: "Contextualizer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, None, 61)]   0           []                               
                                                                                                  
 FC1 (TimeDistributed)          (None, None, 150)    9300        ['Input[0][0]']                  
                                                                                                  
 LSTM1 (Bidirectional)          (None, None, 256)    146688      ['Input[0][0]']                  
                                                                                                  
 dropout_4 (Dropout)            (None, None, 150)    0           ['FC1[0][0]']                    
                                                                                     

KeyboardInterrupt: 

# FC

In [82]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, TimeDistributed, Bidirectional, LSTM, GRU, multiply, Activation
from tensorflow.keras.regularizers import l1

leadfield, pos = util.unpack_fwd(fwd)[1:3]
n_channels, n_dipoles = leadfield.shape
input_shape = (None, None, n_channels)
tf.keras.backend.set_image_data_format('channels_last')

n_dense_units = 300
n_lstm_units = 30
activation_function = "tanh"
batch_size = 32
epochs = 100
dropout = 0.1

def threshold_activation(x):
    return tf.cast(x > 0.3, dtype=tf.float32)

inputs = tf.keras.Input(shape=(None, n_channels), name='Input')
fc1 = TimeDistributed(Dense(n_dense_units, 
            activation=activation_function), 
            name='FC1')(inputs)
direct_out = TimeDistributed(Dense(n_dipoles, 
            activation="linear"),
            name='FC2')(fc1)


model2 = tf.keras.Model(inputs=inputs, outputs=direct_out, name='FC')


model2.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer="adam")

model2.summary()
model2.fit(X, y, epochs=5, batch_size=batch_size, validation_split=0.15)


model2.compile(loss=chamfer(pos), optimizer="adam", metrics=[tf.keras.losses.CosineSimilarity(),])

model2.summary()
model2.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Model: "FC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, None, 61)]        0         
                                                                 
 FC1 (TimeDistributed)       (None, None, 300)         18600     
                                                                 
 FC2 (TimeDistributed)       (None, None, 1284)        386484    
                                                                 
Total params: 405,084
Trainable params: 405,084
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
27/27 [==============================] - 1s 11ms/step - loss: -0.0118 - val_loss: -0.0280
Epoch 2/5
27/27 [==============================] - 0s 13ms/step - loss: -0.0722 - val_loss: -0.0455
Epoch 3/5
27/27 [==============================] - 0s 8ms/step - loss: -0.1024 - val_loss: -0.0577
Epoch 4/5
27/27 [======

KeyboardInterrupt: 

# Eval

In [71]:
import numpy as np
n_samples = 2
settings = dict(duration_of_trial=0.25, number_of_sources=1, extents=(1,20))
sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

X_test = sim_test.eeg_data[0].average().data
X_test = (X_test- X_test.mean()) / X_test.std()
y_test = sim_test.source_data[0].data
y_test /= np.max(np.abs(y_test))

X_test = X_test[np.newaxis]
y_test = y_test[np.newaxis]

X_test = np.swapaxes(X_test, 1,2)
y_test = np.swapaxes(y_test, 1,2)
print(X_test.shape, y_test.shape)

y_hat = model.predict(X_test)[0]
stc = sim_test.source_data[0]
stc.plot(**plot_params)

stc_hat = stc.copy()
stc_hat.data = y_hat.T
stc_hat.plot(**plot_params)


y_hat = model2.predict(X_test)[0]
stc = sim_test.source_data[0]

stc_hat = stc.copy()
stc_hat.data = y_hat.T
stc_hat.plot(**plot_params)


Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 2005.88it/s]


source data shape:  (1284, 25) (1284, 25)


100%|██████████| 2/2 [00:00<00:00, 200.53it/s]


(1, 25, 61) (1, 25, 1284)
